# SPM using Kowiki Dataset

## 01. Dataset Load

- `glob` 모듈을 이용해 kowiki dataset을 로드한다.

In [1]:
from glob import glob

In [2]:
file_lists = glob('../data/kowiki/*/*')
file_lists.sort()

In [8]:
# file_lists

## 02. Data Preprocessing for SPM

- SentencePiece Model의 Input에 맞게끔 데이터를 처리해준다.

- `nltk` 모듈을 이용해 문서를 문장 단위로 분리한다.

- `spm` 모듈의 input을 위해 `spm_input.txt`를 생성한다.

- reformer를 pretraining 시키기 위한 corpus를 pkl로 저장한다.

In [3]:
import re
import json

import sentencepiece as spm

from nltk import sent_tokenize
from tqdm.notebook import tqdm

In [4]:
corpus = []

for file_path in tqdm(file_lists):
    items = []
    with open(file_path, encoding="utf-8") as source:
        raw_text = source.readlines()
        for obj in raw_text:
            text = json.loads(obj)['text']
            text = re.sub('\\n', ' ', text)
            text = re.sub('\\s+', ' ', text)
            items.append(text)
    
    for text in items:
        sents = sent_tokenize(text)
        corpus.extend(sents)

In [15]:
# corpus

In [6]:
import dill

# save corpus for reformer pretraining
with open('../data/corpus/kowiki_corpus.pkl', 'wb') as f:
    dill.dump(corpus, f)

In [17]:
input_file = '../data/spm/spm_input.txt'

with open(input_file, 'w', encoding='utf-8') as f:
    for sent in corpus:
        f.write(f'{sent}\n')

## 03. Train SentencePieceModel

In [19]:
%%time 

vocab_size = 40000
prefix = '200221_kowiki'
cmd = f'--input={input_file} --model_prefix={prefix} --vocab_size={vocab_size}'

spm.SentencePieceTrainer.Train(cmd)

CPU times: user 21min 57s, sys: 4.98 s, total: 22min 2s
Wall time: 5min 33s


True

## 04. Check SPM

In [20]:
import sentencepiece as spm

In [21]:
spm_path = '../data/spm/200221_kowiki.model'

sp = spm.SentencePieceProcessor()
sp.Load(f'{spm_path}')

True

In [23]:
tmp_text = "지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다."

s = sp.EncodeAsPieces(tmp_text)
print(s)

['▁지미', '▁카터', '는', '▁조지아주', '▁섬', '터', '▁카운티', '▁플레', '인', '스', '▁마을에서', '▁태어났다', '.']
